# Optimize
Optimze Gauss-Model Function with Python Profilers

In [1]:
import cProfile
import re
import pstats
from pstats import SortKey
import numpy as np
import scipy.optimize as opt

In [2]:
with open('.fit_area.npy', 'rb') as f:
        ar = np.load(f)

z_values_in = ar.flatten()
print(np.amax(z_values_in))
edge_x = ar.shape[1]
edge_y = ar.shape[0]
    #TO DO: double check if there is some better syntax
x_values = np.repeat(np.array([range(0,edge_x)]), edge_y, axis=0).flatten()
y_values = np.repeat(np.array([range(edge_y-1, -1, -1)]).reshape(edge_y,1), edge_x, axis=1).flatten()


15373


In [3]:
def guess(x, y, z):
    """Estimate starting values from 2D peak data and create Parameters."""
    if x is None or y is None:
        return 1.0, 0.0, 0.0, 1.0, 1.0

    maxx, minx = np.amax(x), np.amin(x)
    maxy, miny = np.amax(y), np.amin(y)
    maxz, minz = np.amax(z), np.amin(z)

    centerx = x[np.argmax(z)]
    centery = y[np.argmax(z)]
    amplitude = (maxz - minz) #quasi height
    sigmax = (maxx-minx)/6.0
    sigmay = (maxy-miny)/6.0
    offset = minz

    return amplitude,centerx,centery,sigmax,sigmay, 0, offset

In [4]:
def twoD_Gaussian( x_y , init):
    amplitude, x_center, y_center, sigma_x, sigma_y, theta, offset = init
    x, y = x_y
    xo = float(x_center)
    yo = float(y_center)    
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    result = offset + amplitude*np.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) + c*((y-yo)**2)))
    return result.ravel()

In [5]:
initial_guess = guess(x_values, y_values, z_values_in)

Analyse optimieren

In [13]:
cProfile.run('re.compile("opt.curve_fit(twoD_Gaussian, (x_values, y_values), z_values_in, p0=initial_guess)")', 'restats')

In [18]:
p = pstats.Stats('restats')
#p.strip_dirs().sort_stats(-1).print_stats()

In [19]:
p.sort_stats(SortKey.CUMULATIVE).print_stats(15)

Tue Jun 28 14:31:00 2022    restats

         688 function calls (675 primitive calls) in 0.001 seconds

   Ordered by: cumulative time
   List reduced from 43 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.001    0.001 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 c:\Python310\lib\re.py:249(compile)
        1    0.000    0.000    0.000    0.000 c:\Python310\lib\re.py:288(_compile)
        1    0.000    0.000    0.000    0.000 c:\Python310\lib\sre_compile.py:759(compile)
        1    0.000    0.000    0.000    0.000 c:\Python310\lib\sre_parse.py:937(parse)
      3/1    0.000    0.000    0.000    0.000 c:\Python310\lib\sre_parse.py:435(_parse_sub)
      3/1    0.000    0.000    0.000    0.000 c:\Python310\lib\sre_parse.py:493(_parse)
        1    0.000    0.000    0.000    0.000 c:\Python310\lib\sre_co

In [21]:
cProfile.run('re.compile("twoD_Gaussian((x_values, y_values), initial_guess)")', 'restats')

In [23]:
p = pstats.Stats('restats')
p.sort_stats(SortKey.CUMULATIVE).print_stats()

Tue Jun 28 14:31:55 2022    restats

         6 function calls in 0.000 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 c:\Python310\lib\re.py:249(compile)
        1    0.000    0.000    0.000    0.000 c:\Python310\lib\re.py:288(_compile)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


